<a href="https://colab.research.google.com/github/vishnu-chand/complex-augmentation/blob/main/dreamAugmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

# @title Dream Augmentations { run: "auto", form-width: "300px" }

# @markdown * Lot of creative ideas originate from dream because, dreams and dreamers do not have any reality barriers.

# @markdown * Similarly following augmentation synthesize image without holding reality, perceptual or context ideas.

# @markdown * Ironically, model train without context or perceptual bias helps to develop better feature representations which can later fine tuned with original images.

# @markdown Download VOC dataset will take some time

import os

if '' or not os.path.exists('/content/db'): # run for only one time
    !pip install albumentations --upgrade
    !git clone https://github.com/vishnu-chand/pixUtils.git
    !pip install xmltodict
    %cd /content/pixUtils
    !git checkout c7be621b64beceea7921eb1e12b9f37573e3887e
    from pixUtils import *
    %cd /content
    downloadDB('wget+http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar', des='/content/db')


from ipywidgets import interact
import albumentations as A
import xmltodict
%cd /content/pixUtils
from pixUtils import *
%cd /content
random.seed(177)
np.random.seed(177)


bgColor = [0, 0, 0]
bgPaths = rglob('/content/db/VOCdevkit/VOC2012/JPEGImages/*.jpg')
yPaths = glob('/content/db/VOCdevkit/VOC2012/SegmentationObject/*.png')


# for i in rglob('/content/db/VOCdevkit/VOC2012/*'):
#     print(rglob(f'{i}/*')[0])


def showImg(winName, imgs, wait=0, figsize=(5, 7)):
    if type(imgs) != list:
        imgs = [imgs]
    for img in imgs:
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            pass
        plt.figure(figsize=figsize)
        plt.imshow(img)
        plt.show()


def extract_bboxes(masks, removeEmptyMask=False):
    bboxes = []
    h, w = masks[0].shape
    for mask in masks:
        yindices = np.where(np.any(mask, axis=0))[0]
        xindices = np.where(np.any(mask, axis=1))[0]
        x0y0x1y1 = 0, 0, 0, 0
        if yindices.shape[0]:
            x0, x1 = yindices[[0, -1]]
            y0, y1 = xindices[[0, -1]]
            x0y0x1y1 = x0 / w, y0 / h, (x1 + 1) / w, (y1 + 1) / h
            bboxes.append(x0y0x1y1)
        elif not removeEmptyMask:
            bboxes.append(x0y0x1y1)
    return bboxes


def read_label(file, dtype=np.int32):
    f = Image.open(file)
    try:
        img = f.convert('P')
        img = np.array(img, dtype=dtype)
    finally:
        if hasattr(f, 'close'):
            f.close()
    if img.ndim == 2:
        return img
    elif img.shape[2] == 1:
        return img[:, :, 0]
    else:
        raise ValueError("Color image can't be accepted as label image.")


def image_wise_to_instance_wise(label_img, inst_img, removeEmptyMask=False):
    voc_bbox_label_names = ('aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor')
    mask = []
    label = []
    inst_ids = np.unique(inst_img)
    for maskIx, inst_id in enumerate(inst_ids[inst_ids != -1]):
        msk = inst_img == inst_id
        lbl = label_img[msk][0] - 1
        assert inst_id != -1
        assert lbl != -1
        msk = cv2.morphologyEx(np.uint8(msk), cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=1)
        mask.append(msk)
        label.append([voc_bbox_label_names[lbl], maskIx])
    label = [(x0, y0, x1, y1, label, maskIx) for (x0, y0, x1, y1), (label, maskIx) in zip(extract_bboxes(mask, removeEmptyMask), label)]
    return mask, label


def getData(yPath, getSegmentationData, getInstanceData, getBboxData):
    # print(yPath)
    img = cv2.imread(yPath.replace('SegmentationObject', 'JPEGImages').replace('.png', '.jpg'))
    inst_img = read_label(yPath, dtype=np.int32)
    segPath = yPath.replace('SegmentationObject', 'SegmentationClass')
    label_img = read_label(segPath, dtype=np.int32)
    # label_img, inst_img = readPkl('/home/hippo/Downloads/d.pkl', [label_img, inst_img])
    inst_img[inst_img == 0] = -1
    inst_img[inst_img == 255] = -1
    label_img[label_img == 255] = -1
    masks, bboxs = image_wise_to_instance_wise(label_img, inst_img, removeEmptyMask=True)
    res = dict(image=img, bboxes=[])
    if getSegmentationData and not getInstanceData:
        res['mask'] = cv2.imread(segPath)
    if getBboxData:
        if not getInstanceData:
            bboxs = [(x0, y0, x1, y1, label) for x0, y0, x1, y1, label, maskId in bboxs]
        res['bboxes'] = bboxs
    if getInstanceData:
        res['masks'] = masks
    return res


def getDispXY(t):
    img, mask, masks, bboxes = t['image'].copy(), t.get('mask'), t.get('masks'), t.get('bboxes', [])
    for b in bboxes:
        # print(b)
        x0, y0, x1, y1 = b[:4]
        h, w = img.shape[:2]
        x0, y0, x1, y1 = x0 * w, y0 * h, x1 * w, y1 * h
        color = np.random.uniform(100, 255, 3)
        cv2.rectangle(img, (int(x0), int(y0)), (int(x1), int(y1)), list(color), 3)
    if mask is not None:
        mask = mask.copy()
    if masks is not None:
        imH, imW = masks[0].shape[:2]
        mask = np.zeros((imH, imW, 3), 'u1')
        for m in masks:
            mask[m != 0] = np.random.uniform(0, 255, 3).astype('u1')
        mask = mask.copy()
    return img, mask


print("\n\nInitial setup completed")
t = getData(random.choice(yPaths), getSegmentationData=' ', getInstanceData=' ', getBboxData=' ')
img, mask = getDispXY(t)
showImg("", [img, mask])

In [ ]:
# @title Interleave Augmentation [choose both values less than 0.15] { run: "auto", form-width: "300px" }
# @markdown * Take two images i1 and i2 and replace nRows and mCols of i1 with i2 pixel

# @markdown * This emulate pixel level occultion and helps model to reject unwanted image connections

getSegmentationData = True  # @param {type:'boolean'}
getInstanceData = True  # @param {type:'boolean'}
getBboxData = True  # @param {type:'boolean'}
maxInterleaveWidthPc = 0.08  # @param {type:"slider", min:0, max:1, step:0.01}
maxInterleaveHeightPc = 0.08  # @param {type:"slider", min:0, max:1, step:0.01}

imgPath = '/content/db/VOCdevkit/VOC2012/SegmentationObject/2009_003504.png'


def InterleaveAug(maxInterleaveHeightPc=0.03, maxInterleaveWidthPc=0.03, p=0.5):
    '''
    maxInterleaveHeightPc: (0.02, 0.15) if maxInterleaveHeightPc=.5, 50% of image will be replace with interleaveImg 
    maxInterleaveWidthPc: (0.02, 0.15) if maxInterleaveWidthPc=.1, 10% of image will be replace with interleaveImg
    interleaveImg: image whose pixels will be used for interleaving 
    '''

    def __prcs(image, interleaveImg, force_apply=False, **kw):
        if force_apply or random.random() < p:
            imH, imW = image.shape[:2]
            i = random.sample(range(imH), int(imH * maxInterleaveWidthPc))
            j = random.sample(range(imW), int(imW * maxInterleaveHeightPc))
            interleaveImg = cv2.resize(interleaveImg, (imW, imH))
            image[i] = interleaveImg[i]
            image[:, j] = interleaveImg[:, j]
        return dict(image=image, **kw)

    return __prcs


def interleaveAug(transformT, bgT, data):
    x = data['image'].copy()
    t = transformT(interleaveImg=interleaveImg, **data)
    xt, yt = getDispXY(t)
    disp = [x, interleaveImg, xt]
    if yt is not None:
        disp.append(yt)
    showImg('augumentData_img', photoframe(disp), 0, figsize=(12, 12))


transformT = A.Compose([
    InterleaveAug(maxInterleaveHeightPc, maxInterleaveWidthPc, p=1),
], bbox_params=A.BboxParams('albumentations', min_visibility=.05))
bgT = A.Compose([])
interleaveImg = bgT(image=cv2.imread(random.choice(bgPaths)))['image']
data = getData(imgPath, getSegmentationData, getInstanceData, getBboxData)

interleaveAug(transformT, bgT, data)

In [ ]:
# @title Change or Blur Background Augmentation { run: "auto", form-width: "300px" }
# @markdown * Lets say you have to detect person in middle of railway track or level crossing.
# @markdown * But there is very small data to train a model
# @markdown * In such case, we can take indoor person data and replace background with tracks and train the model
# @markdown * Later fine-tune model with small orginal dataset
# @markdown * Note: for good results ensure bgImg has no person

useRandomImgs = False  # @param {type:'boolean'}
getSegmentationData = True  # @param {type:'boolean'}
getInstanceData = True  # @param {type:'boolean'}
getBboxData = True  # @param {type:'boolean'}


def ChangeOrBlurBackground(bgValue=0, minAlpha=0.75, p=0.5):
    '''
    bgValue: pixel value which replace background, those pixels will be replace with bgImg
    minAlpha: (0.75 to 1.0); will crease superimposition of two image and bgImg
            mixAlpha = np.random.uniform(minAlpha, 1)
            image = cv2.addWeighted(image, mixAlpha, bgImg, 1 - mixAlpha, 0)
    bgImg: image whose pixels will be used to replace the background, if the value is None it will blur current background
    '''

    bgValue = np.array(bgValue)

    def __prcs(image, mask=None, masks=None, bboxes=None, bgImg=None, force_apply=False, **kw):
        if force_apply or random.random() < p:
            if bgImg is None:
                bgImg = cv2.blur(image, (int(random.uniform(11, 21)), int(random.uniform(11, 21))))
            else:
                imH, imW = image.shape[:2]
                bgImg = cv2.resize(bgImg, (imW, imH))
            if masks is not None:
                image = __cutMask(image, masks, bgImg, isMasks=True)
            elif mask is not None:
                image = __cutMask(image, mask, bgImg, isMasks=False)
            elif bboxes:
                image = __cutBbox(image, bboxes, bgImg)
            else:
                raise Exception("""
                Either mask or masks or bboxes should be passed
                """)
        return dict(image=image, mask=mask, masks=masks, bboxes=bboxes, bgImg=bgImg, **kw)

    def __cutMask(image, mask, bgImg, isMasks):
        if isMasks:
            roi = mask[0] > 0
            for m in mask[1:]:
                roi |= m > 0
            roi = ~roi
        else:
            if len(mask.shape) == 3:
                roi = np.all(mask == bgValue, axis=-1)
            else:
                roi = mask == bgValue
        mixAlpha = np.random.uniform(minAlpha, 1)
        image = cv2.addWeighted(image, mixAlpha, bgImg, 1 - mixAlpha, 0)
        image[roi] = bgImg[roi]
        return image

    def __cutBbox(image, bboxes, bgImg):
        imH, imW = image.shape[:2]
        for x0, y0, x1, y1, label in bboxes:
            # assert 0 <= x0 < 1 and 0 <= y0 < 1 and 0 < x1 <= 1 and 0 < y1 <= 1
            x0, y0, x1, y1 = int(x0 * imW), int(y0 * imH), int(x1 * imW), int(y1 * imH)
            mixAlpha = np.random.uniform(minAlpha, 1)
            bgImg[y0:y1, x0:x1] = cv2.addWeighted(image[y0:y1, x0:x1], mixAlpha, bgImg[y0:y1, x0:x1], 1 - mixAlpha, 0)
        image = bgImg
        return image

    return __prcs


def changeOrBlurBackground(imgPath, replaceBgs, minAlpha):
    transformT = A.Compose([
        ChangeOrBlurBackground(bgValue=0, minAlpha=minAlpha, p=1)
    ], bbox_params=A.BboxParams('albumentations', min_visibility=.05))
    data = getData(imgPath, getSegmentationData, getInstanceData, getBboxData)
    t = transformT(bgImg=bgImg, **data)
    xt, yt = getDispXY(t)
    return xt, yt


imgPath = '/content/db/VOCdevkit/VOC2012/SegmentationObject/2009_003504.png'
replaceBgs = ['/content/db/VOCdevkit/VOC2012/JPEGImages/2011_001782.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2007_004705.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2008_001349.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2012_002277.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2010_001982.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2010_004546.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2008_008097.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2008_007266.jpg', '/content/db/VOCdevkit/VOC2012/JPEGImages/2011_000612.jpg']

xs, ys = [], []
bgT = A.Compose([])
if useRandomImgs:
    imgPath = random.choice(yPaths)
for bgImg in replaceBgs:
    if useRandomImgs:
        bgImg = random.choice(bgPaths)
        # print(bgImg)
    bgImg = bgT(image=cv2.imread(bgImg))['image']
    x, y = changeOrBlurBackground(imgPath, replaceBgs, minAlpha=.75)
    xs.append(x)
    if y is not None:
        ys.append(y)
showImg('', photoframe(xs), 0, figsize=(15, 15))
if ys:
    showImg('', photoframe(ys), 0, figsize=(15, 15))


In [ ]:
# @title Image Collage augmentation { run: "auto", form-width: "300px" }
# @markdown * Randomly select 4 images and do random geometric transformation and create a collage
# @markdown * This will create a trainding dataset with densely packed features in a small receptive field.
# @markdown * And helps the model to achieve better accuracy (avoid using such techniques in fine tuning stage)

getSegmentationData = True  # @param {type:'boolean'}
getBboxData = True  # @param {type:'boolean'}


def ImageCollage(collageT):
    '''
    collageT: A A.Compose object. Before applying collage, every sub-image will undergo this transform  
    '''
    r2c2 = {0: [[0.5, 0.0], [0.5, 0.0], [0.5, 0.0], [0.5, 0.0]],
            1: [[0.5, 0.5], [0.5, 0.0], [0.5, 0.5], [0.5, 0.0]],
            2: [[0.5, 0.0], [0.5, 0.5], [0.5, 0.0], [0.5, 0.5]],
            3: [[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]}
    r2c1 = {0: [[1.0, 0.0], [0.5, 0.0], [1.0, 0.0], [0.5, 0.0]],
            1: [[1.0, 0.0], [0.5, 0.5], [1.0, 0.0], [0.5, 0.5]]}
    r1c2 = {0: [[0.5, 0.0], [1.0, 0.0], [0.5, 0.0], [1.0, 0.0]],
            1: [[0.5, 0.5], [1.0, 0.0], [0.5, 0.5], [1.0, 0.0]]}

    def __prcs(datas):
        if len(datas) == 4:
            shiftBy, nRow, nCol = r2c2, 2, 2
        else:
            shiftBy, nRow, nCol = random.choice([[r2c1, 2, 1], [r1c2, 1, 2]])
        for d in datas:
            if d.get('masks'):
                raise Exception("""
                masks is not supported
                """)
        datas = [collageT(**data) for data in datas]
        image = __mergeData(datas, 'image', nCol)
        mask = __mergeData(datas, 'mask', nCol)
        collageData = dict(image=image, mask=mask, bboxes=[])
        collageData = __collageBox(datas, collageData, shiftBy)
        return collageData

    def __collageBox(datas, collageData, shiftBy):
        for quater, data in enumerate(datas):
            bboxes = data.get('bboxes', [])
            try:
                for x0, y0, x1, y1, label in bboxes:
                    # assert 0 <= x0 < 1 and 0 <= y0 < 1 and 0 < x1 <= 1 and 0 < y1 <= 1
                    x0, y0, x1, y1 = [m * x + c for x, (m, c) in zip([x0, y0, x1, y1], shiftBy[quater])]
                    collageData['bboxes'].append([x0, y0, x1, y1, label])
            except:
                raise Exception(f"""
                        expected albumentations format [x_min, y_min, x_max, y_max, label]
                        all values should be in the range of 0 to  1
                        """)
        return collageData

    def __mergeData(datas, key, nCol):
        imCol, imRow = [], []
        for ix, d in enumerate(datas, -1):
            i = d.get(key)
            if i is not None:
                imCol.append(i)
                if ix % nCol == 0:
                    imRow.append(cv2.hconcat(imCol))
                    imCol = []
        img = cv2.vconcat(imRow)
        return img

    return __prcs


def collageTest():
    bgColor = [0, 0, 0]
    subImgWidth, subImgHeight = 384, 384
    collageT = A.Compose([
        A.Resize(height=subImgHeight, width=subImgWidth, p=1),
        A.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, value=bgColor, p=.5,
                           scale_limit=[-.4, -.2], shift_limit=[-.4, -.2], rotate_limit=15),
        A.HorizontalFlip(),
    ], bbox_params=A.BboxParams('albumentations', min_visibility=.05))

    collageIt = ImageCollage(collageT)
    res = []
    for i in range(3):
        while 1:
            data = getData(random.choice(yPaths), getSegmentationData, False, getBboxData)
            res.append(data)
            if len(res) == 2 and random.random() < .5 or len(res) == 4:
                t = collageIt(res)
                img, mask = getDispXY(t)
                if mask is not None:
                    img = cv2.hconcat([img, mask])
                showImg('augumentData_image', img, 0, figsize=(15, 15))
                res, aug = [], False
                break


collageTest()

In [ ]:
# @title CopyPaste Augmentation { run: "auto", form-width: "300px" }

from skimage.filters import gaussian


def image_copy_paste(img, paste_img, alpha, alphaMix, blend=True, sigma=1):
    if alpha is not None:
        if blend:
            alpha = gaussian(alpha, sigma=sigma, preserve_range=True)
        img_dtype = img.dtype
        alpha = alpha[..., None] * alphaMix
        img = paste_img * alpha + img * (1 - alpha)
        img = img.astype(img_dtype)
    return img


def masks_copy_paste(masks, paste_masks, alpha):
    if alpha is not None:
        # eliminate pixels that will be pasted over
        masks = [
            np.logical_and(mask, np.logical_xor(mask, alpha)).astype(np.uint8) for mask in masks
        ]
        masks.extend(paste_masks)
    return masks


def bboxes_copy_paste(bboxes, paste_bboxes, masks, paste_masks, alpha):
    if paste_bboxes is not None:
        masks = masks_copy_paste(masks, paste_masks=[], alpha=alpha)
        adjusted_bboxes = extract_bboxes(masks)
        # only keep the bounding boxes for objects listed in bboxes
        mask_indices = [box[-1] for box in bboxes]
        adjusted_bboxes = [adjusted_bboxes[idx] for idx in mask_indices]
        # append bbox tails (classes, etc.)
        adjusted_bboxes = [bbox + tail[4:] for bbox, tail in zip(adjusted_bboxes, bboxes)]
        # adjust paste_bboxes mask indices to avoid overlap
        max_mask_index = len(masks)
        paste_mask_indices = [max_mask_index + ix for ix in range(len(paste_bboxes))]
        paste_bboxes = [pbox[:-1] + (pmi,) for pbox, pmi in zip(paste_bboxes, paste_mask_indices)]
        adjusted_paste_bboxes = extract_bboxes(paste_masks)
        adjusted_paste_bboxes = [apbox + tail[4:] for apbox, tail in zip(adjusted_paste_bboxes, paste_bboxes)]
        bboxes = adjusted_bboxes + adjusted_paste_bboxes
    return bboxes


def keypoints_copy_paste(keypoints, paste_keypoints, alpha):
    # remove occluded keypoints
    if alpha is not None:
        visible_keypoints = []
        for kp in keypoints:
            x, y = kp[:2]
            tail = kp[2:]
            if alpha[int(y), int(x)] == 0:
                visible_keypoints.append(kp)
        keypoints = visible_keypoints + paste_keypoints
    return keypoints


def CopyPaste(blend=True, sigma=3, pct_objects_paste=1.0, max_paste_objects=np.inf, minAlpha=.75, p=0.5):
    '''
    blend = True and sigma: (0.0 to 5.0) the border of objects will be blur to have smooth paste effect
    pct_objects_paste: (0.0 to 1.0) if pct_objects_paste = 0.5 half of objects will randomly choose for copying
    max_paste_objects: (0 to inf) if max_paste_objects = 3 at max 3 objects will be randomly pasted on src image
    minAlpha: (0.75 to 1.0); will crease superimposition of two image and bgImg
        mixAlpha = np.random.uniform(minAlpha, 1)
        image = cv2.addWeighted(image, mixAlpha, bgImg, 1 - mixAlpha, 0)
    paste_image = image used to paste
    paste_masks = its corresponding masks (mask/paste_mask is not supported please split as masks it before using copy paste)
    paste_bboxes (optional) bounding boxes
    '''

    def __prcs(image, masks, mask=None, bboxes=None, paste_image=None, paste_masks=None, paste_bboxes=None, force_apply=False, **kw):
        if mask is not None:
            raise Exception(f"""
            mask is not supported please split it and pass as masks argument 
            """)
        if force_apply or random.random() < p:
            if paste_bboxes and not paste_masks:
                raise Exception("""
                For proper working of paste_bboxes paste_masks are required.
                Please try other augmentation like ReplaceBackground""")
            ix = range(len(paste_bboxes)) if paste_bboxes else range(len(paste_masks))
            n_objects = len(ix)
            n_select = min(int(n_objects * pct_objects_paste), max_paste_objects)
            ix = np.random.choice(ix, size=n_select, replace=False)
            if paste_bboxes:
                paste_bboxes = [paste_bboxes[i] for i in ix]
                ix = [bbox[-1] for bbox in paste_bboxes]
            alpha = None
            if paste_masks:
                paste_masks = [paste_masks[i] for i in ix]
                alpha = paste_masks[0] > 0
                for mask in paste_masks[1:]:
                    alpha += mask > 0

            bboxes = bboxes_copy_paste(bboxes, paste_bboxes, masks, paste_masks, alpha)
            image = image_copy_paste(image, paste_image, alpha, random.uniform(minAlpha, 1), blend=blend, sigma=sigma)
            masks = masks_copy_paste(masks, paste_masks, alpha)
        return dict(image=image, masks=masks, bboxes=bboxes, paste_image=paste_image, paste_masks=paste_masks, paste_bboxes=paste_bboxes, **kw)

    return __prcs


# @markdown  Code partially copied from https://github.com/conradry/copy-paste-aug

# @markdown * [Simple Copy-Paste is a Strong Data Augmentation Method for Instance Segmentation](https://arxiv.org/abs/2012.07177v2)

In [ ]:
# @title Copy Paste Augmentation  { run: "auto", form-width: "300px" }
# @markdown * Randomly choose image x1 with instance mask y1, paste image px1 with mask py1 let's say py1 has n objects
# @markdown * Randomly choose m object from py1 (m <= n), crop m objects from px1 using mask py1 and paste it in x1 and update mask y1 accordingly
# @markdown * Correct newly generated image and mask x1, y1 remove fully occluded objects and update bounding box of partially occluded objects

useRandomImgs = False  # @param {type:'boolean'}
getBboxData = True  # @param {type:'boolean'}
getSegmentationData = False
getInstanceData = True

transformT = A.Compose([
    A.Resize(384, 384),
    CopyPaste(p=1, minAlpha=0.75),
], bbox_params=A.BboxParams('albumentations', min_visibility=.05))

pasteT = A.Compose([
    A.Resize(384, 384),
], bbox_params=A.BboxParams('albumentations', min_visibility=.05))

imgPath = '/content/db/VOCdevkit/VOC2012/SegmentationObject/2009_003504.png'
pastePaths = ['/content/db/VOCdevkit/VOC2012/SegmentationObject/2010_002413.png', '/content/db/VOCdevkit/VOC2012/SegmentationObject/2011_001855.png', '/content/db/VOCdevkit/VOC2012/SegmentationObject/2008_001358.png']

if useRandomImgs:
    imgPath = random.choice(yPaths)
    print(imgPath)

for pastePath in pastePaths:
    if useRandomImgs:
        pastePath = random.choice(yPaths)
        print(pastePath)
    data = getData(imgPath, getSegmentationData, getInstanceData, getBboxData)
    pasteData = getData(pastePath, getSegmentationData, getInstanceData, getBboxData)
    x, y = getDispXY(data)
    px, py = getDispXY(pasteData)
    pasteData['masks'] = [m.astype('u1') for m in pasteData['masks']]
    data['masks'] = [m.astype('u1') for m in data['masks']]
    pasteData = pasteT(**pasteData)
    for k, v in pasteData.items():
        data[f"paste_{k}"] = v
    t = transformT(**data)
    xt, yt = getDispXY(t)
    showImg('augumentData_img', photoframe([x, px, xt, y, py, yt], nRow=2), 0, figsize=(25, 25))


In [ ]:
# @title Let's create a complex augmentation pipelines { run: "auto", form-width: "300px" } 
# @markdown * Using
# @markdown * CopyPaste
# @markdown * InterleaveAug
# @markdown * RandomSnow
# @markdown * RandomRain
# @markdown * RandomShadow
# @markdown * RandomSunFlare
# @markdown * Geometric Transforms (ShiftScaleRotate)
# @markdown * RandomBrightnessContrast and RGBShift

useRandomImgs = False  # @param {type:'boolean'}
getSegmentationData = True  # @param {type:'boolean'}
getInstanceData = True  # @param {type:'boolean'}
getBboxData = True  # @param {type:'boolean'}

bgT = A.Compose([
    A.Resize(width=640, height=640, p=1),
    A.ShiftScaleRotate(border_mode=cv2.BORDER_WRAP, value=bgColor, p=.5),
    A.RandomBrightnessContrast(p=.3),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=.3),
    A.RandomGridShuffle(p=.5),
])

interleaveT = A.Compose([
    A.Resize(width=640, height=640, p=1),
    A.ShiftScaleRotate(border_mode=cv2.BORDER_WRAP, value=bgColor, p=.5),
    A.RandomBrightnessContrast(p=.3),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=.3),
])

pasteT = A.Compose([
    A.Resize(width=640, height=640, p=1),
    A.RandomBrightnessContrast(p=.3),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=.3),
    A.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, value=bgColor, scale_limit=[-.6, -.2], shift_limit=[-.4, .4], rotate_limit=15, p=1),
    A.HorizontalFlip(p=.5),
], bbox_params=A.BboxParams('albumentations', min_visibility=.05))

srcT = A.Compose([
    A.Resize(width=640, height=640, p=1),
    A.RandomBrightnessContrast(p=.3),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=.3),
    InterleaveAug(0.01, 0.01, p=0.2),
    CopyPaste(minAlpha=.75, p=1),
    # ChangeOrBlurBackground(p=1),

    # A.Transpose(p=.5),
    # A.HorizontalFlip(p=.5),
    # A.VerticalFlip(p=.5),
    # A.RandomSnow(),
    # A.RandomRain(),
    A.RandomShadow(),
    # A.RandomSunFlare(),
], bbox_params=A.BboxParams('albumentations', min_visibility=.05))

for _ in range(3):
    srcPath = random.choice(yPaths)
    pastePath = random.choice(yPaths)
    bgPath = random.choice(bgPaths)
    interleavePath = random.choice(bgPaths)
    print("note, instance segmentation colors are random")
    print("srcPath:       ", srcPath)
    print("pastePath:     ", pastePath)
    print("interleavePath:", interleavePath)

    data = getData(imgPath, getSegmentationData, getInstanceData, getBboxData)
    pasteData = getData(pastePath, getSegmentationData, getInstanceData, getBboxData)

    x, y = getDispXY(data)
    px, py = getDispXY(pasteData)
    pasteData['masks'] = [m.astype('u1') for m in pasteData['masks']]
    data['masks'] = [m.astype('u1') for m in data['masks']]
    pasteData = pasteT(**pasteData)
    for k, v in pasteData.items():
        data[f"paste_{k}"] = v
    data['bgImg'] = bgT(image=cv2.imread(bgPath))['image']
    data['interleaveImg'] = interleaveT(image=cv2.imread(interleavePath))['image']

    t = srcT(**data)
    xt, yt = getDispXY(t)
    showImg('augumentData_img', photoframe([x, px, xt, y, py, yt], nRow=2), 0, figsize=(25, 25))